# Teams Over Time

This notebook explains the 'horizontal' slice of how sentiment analysis depicts a particular team's gamethreads over multiple weeks.

## Imports

In [1]:
# Path hack
import os
# change directory from the current Analysis folder to the top level folder for easier navigation
os.chdir('../')
# confirm we're at /RedditTextAnalysis
print(os.getcwd())

# note that the %load_ext autoreload line only needs to be be run once
%load_ext autoreload

/Users/prcork/MiscDataProjects/collaboration/RedditTextAnalysis


In [2]:
# by including this autoreload command, we only need to re-import if we make/save changes to the original py file
%autoreload
from Functions.nfl_gamethreads import nfl_gamethreads

In [3]:
import pandas as pd
import sqlite3
import altair as alt

## Query Comments

In [10]:
with sqlite3.connect("nfl_gamethreads.db") as conn:

    # pull in all comments from weeks 4-9, 2021
    comments_df = pd.read_sql_query('''SELECT comments.comment_id, comments.author, comments.body, comments.author_flair, comments.upvotes, comments.utc_time, 
                                              gamethreads.week, gamethreads.season
                                    FROM comments 
                                    INNER JOIN gamethreads ON comments.submission_id = gamethreads.submission_id 
                                    WHERE gamethreads.week <= 9''', conn)

In [12]:
comments_df = nfl_gamethreads.analyze_text(comments_df, 'body')

In [23]:
comments_df

,comment_id,author,body,author_flair,upvotes,utc_time,week,season,polarity,subjectivity
0,hej5m66,Piano_Fingerbanger,Boo both teams! \n\nLet's get a tie!,buccaneers,2,1.632786e+09,3,2021,0.000000,0.000000
1,hej5n9d,None,[deleted],none,5,1.632786e+09,3,2021,0.000000,0.000000
2,hej5n9g,TheBigSm0ke,Good measuring stick game for Hurts. Let’s see...,none,1,1.632786e+09,3,2021,0.150000,0.500000
3,hej5ner,harshalax31,Hopefully the streak of amazing prime time gam...,49ers,2,1.632786e+09,3,2021,0.600000,0.900000
4,hej5nmk,Saitsu,"Oh I'm ready, BRING ON THE TOXICITY!\n\nAs for...",none,3,1.632786e+09,3,2021,-0.183333,0.366667
...,...,...,...,...,...,...,...,...,...,...
636998,hjgrea1,Lieutenant_Meeper,Goddamn the most recent Super Bowl win really ...,broncos,1,1.636146e+09,9,2021,0.438889,0.452778
636999,hjgwm8t,andrewfnluck,It’s all in fun. I would prefer a couple of co...,colts,1,1.636148e+09,9,2021,0.400000,0.433333
637000,hjh8nqv,None,[deleted],none,1,1.636152e+09,9,2021,0.000000,0.000000
637001,hjh9rbz,Zivmovic,I forgot the scientists that have made vaccine...,giants,1,1.636153e+09,9,2021,0.338542,1.000000


In [29]:
week_sum = comments_df[['polarity','subjectivity','week', 'author_flair']].groupby(['week', 'author_flair']).mean()

In [32]:
saints = week_sum.query("author_flair == 'saints'").copy()
titans = week_sum.query("author_flair == 'titans'").copy()

In [33]:
saints

,,polarity,subjectivity
week,author_flair,,
3,saints,0.036483,0.354191
4,saints,0.037606,0.350719
5,saints,0.013548,0.352109
6,saints,0.035295,0.340928
7,saints,0.028573,0.345988
8,saints,0.025710,0.358867
9,saints,0.017860,0.356629


In [34]:
titans

,,polarity,subjectivity
week,author_flair,,
3,titans,0.026249,0.359512
4,titans,0.035214,0.366656
5,titans,0.052176,0.373449
6,titans,0.023104,0.356951
7,titans,0.049635,0.378165
8,titans,0.004190,0.391640
9,titans,0.055566,0.358477
